In [2]:
## History management- It is required to manage the history of chat since unlimited chat history can lead to overflow of memory.

from langchain_core.messages import SystemMessage,trim_messages
from langchain_core.messages import HumanMessage, AIMessage

import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
load_dotenv()

groq_api_key=os.getenv("GROQ_API_KEY")

model=ChatGroq(model="llama-3.1-8b-instant",groq_api_key=groq_api_key)

trimmer =trim_messages(
    max_tokens=40,
    strategy="last",
    token_counter=model, 
    include_system=True,
    allow_partial=False,
    start_on="human"
)

messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="Hi, I'm bob"),
    AIMessage(content="Hello Bob! How can I assist you today?"),
    HumanMessage(content="I like vanilla ice cream."),
    AIMessage(content="Vanilla is a classic choice! Do you have a favorite brand?"),
    HumanMessage(content="what's 2+2"),
    AIMessage(content="2+2 is 4."),
    HumanMessage(content="Thanks!"),
    AIMessage(content="You're welcome!")
]

trimmer.invoke(messages)


[SystemMessage(content='You are a helpful assistant.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content="what's 2+2", additional_kwargs={}, response_metadata={}),
 AIMessage(content='2+2 is 4.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Thanks!', additional_kwargs={}, response_metadata={}),
 AIMessage(content="You're welcome!", additional_kwargs={}, response_metadata={})]

In [3]:
from operator import itemgetter ## itemgetter is a function that returns a callable object that fetches the given item(s) from its operand.

from langchain_core.runnables import RunnablePassthrough ##RunnablePassthrough is a class that simply passes the input to the output without any modification.
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder


prompt=ChatPromptTemplate.from_messages([
    ("system","You are a helpful assistant that helps people find information."),
    MessagesPlaceholder(variable_name="messages")])
chain=(
    RunnablePassthrough().assign(messages=itemgetter("messages")|trimmer) ## Assigning the trimmed messages to the chain. and itemgetter is used to get the messages from the input dictionary.
    | prompt
    | model
)
response=chain.invoke({
    "messages": messages + [HumanMessage(content="what math problem did i asked")],
    "language": "English"
}
)
response.content

"You didn't ask a math problem. Our conversation just started with a greeting. If you'd like to ask a math question or problem, I'm here to help!"

In [ ]:
## Lets wrap this into message history
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory ## BaseChatMessageHistory is the base class for chat history.It is used to define the type of chat history.For example, we can use it to define the type of chat history as ChatMessageHistory.
from langchain_core.runnables import RunnableWithMessageHistory ## RunnableWithMessageHistory is used to create a runnable with message history.It is used to create a runnable that can store the chat history.

store={} ## store is a dictionary that will be used to store the chat history for different sessions.Dictionary here in simple words is a key value pair where key is session id and value is chat history.
def get_session_history(session_id: str) -> BaseChatMessageHistory: ## get_session_history is a function that takes session_id as input and returns the chat history for that session_id.
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory( ## with_message_history is an instance of RunnableWithMessageHistory class.It is used to create a runnable that can store the chat history.
    chain,
    get_session_history,
    input_messsage_key="messages",
)
config={"configurable":{"session_id":"chat5"}}